## **Turi Activity Classifier**

Welcome to the activity classification model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load activity training data.
2. Build an activity classification model.
3. Convert the model to CoreML format and sae it to the Saves framework.

The example is based on [Turi Create's Activity Classifier](https://apple.github.io/turicreate/docs/userguide/activity_classifier/).

---

Execute each cell one-by-one, by selecting the cell and do one of the following:

-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# Import necessary libraries
from skafossdk import *

from utilities.dependencies import install
install(timeout=500, retries=2) # check to make sure dependencies are installed

import turicreate as tc
from utilities.load_data import ActivityData
import utilities.save_models as sm

In [ ]:
ska = Skafos() # initialize Skafos

### 1. **Load the data**
The data for this example comes from the open source HAPT dataset, consisting of activity-labeled experiments and motion-sensory readings from an accelerometer and gyroscope. The data is loaded using a separate module `utilities.load_data`. The data is randomly split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.

In [ ]:
# Load data from a hosted s3 bucket
activity_data = ActivityData()
data = activity_data.get_data() 

In [ ]:
# Make a train-test split such that each data slice contains whole "Experiment" chunks
train_data, test_data = tc.activity_classifier.util.random_split_by_session(data, session_id='Experiment', fraction=0.8)

In [ ]:
# Take a look at the data
train_data.head(5)

### 2. **Build the model**
We use the `tc.activity_classifier.create` function and specify the data, target variable, session_id, and a few other arguments needed to properly train the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/activity_classifier/).

In [ ]:
# Create an activity classifier
model = tc.activity_classifier.create(
    dataset=train_data,
    session_id='Experiment',
    target='Activity',
    prediction_window=50
)

---
**Prediction Window** = *number_seconds_between_predictions* x *sample_frequency*

-  *number_seconds_between_predictions* is how many seconds you want between each prediction. Varies by use-case.
-  *sample_frequency* is how many sensor readings are taken each second (by the application).

For this example, the motion-sensor data from HAPT was collected at 50Hz (or 50 times per second). If we want our model to give an activity prediction every 1 second, we should set the prediction window to 50 ~ (1 * 50).

In [ ]:
# Find 3 seconds of walking data constrained to a single experiment where walking took place
exp_id_with_walking = test_data[test_data['Activity'] == 'walking']['Experiment'].value_counts()[0]['value']
walking_3_seconds = test_data[(test_data['Activity'] == 'walking') & (test_data['Experiment'] == exp_id_with_walking)][:150]

# Find 10 seconds of sitting data constrained to a single experiment where sitting took place
exp_id_with_sitting = test_data[test_data['Activity'] == 'sitting']['Experiment'].value_counts()[0]['value']
sitting_10_seconds = test_data[(test_data['Activity'] == 'sitting') & (test_data['Experiment'] == exp_id_with_sitting)][:500]

In [ ]:
# Check if the model properly classifies each second as walking or something else
model.predict(walking_3_seconds, output_frequency='per_window')

In [ ]:
# Check if the model properly classifies each second as sitting or something else
model.predict(sitting_10_seconds, output_frequency='per_window')

### 3. **Save the model**
Once your model has been created, it must be saved to the Skafos framework via the code below. This will trigger a push to your mobile app.

In [ ]:
## Export to coreml

# Specify the CoreML model name
coreml_model_name = 'ActivityClassifier.mlmodel'
# Specify the name of the compressed model we save to Skafos
compressed_model_name = coreml_model_name + ".gz" 

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Compress the model
compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
sm.skafos_save_model(
    skafos=ska,
    model_name=compressed_model_name,
    compressed_model=compressed_model,
    permissions='public'
)